In [4]:
%%manim -qh -v WARNING SHMwithVoiceover
from manim import *
from manim_voiceover import VoiceoverScene
from manim_voiceover.services.gtts import GTTSService
import numpy as np
import json

class SHMwithVoiceover(VoiceoverScene):
    def construct(self):
        # --- Load storyboard JSON ---
        with open("metadata_ke_frequency.json", "r") as f:
            storyboard = json.load(f)

        # Initialize GTTS voice service
        self.set_speech_service(GTTSService(lang=storyboard["settings"]["speech_language"], speed=1.0))

        # --- Title ---
        with self.voiceover(text=storyboard["scenes"][0]["narration_segments"][0]["text"]) as tracker:
            title = Text("Simple Harmonic Motion: Energy Variation", color=YELLOW).scale(0.8).to_edge(UP)
            self.play(Write(title), run_time=min(2, tracker.duration))

        # --- Reference axis and labels ---
        axis = NumberLine(x_range=[-4, 4, 1], include_numbers=False, color=GRAY).shift(DOWN * 0.5)

        with self.voiceover(text=storyboard["scenes"][0]["narration_segments"][1]["text"]) as tracker:
            eq_point = Dot(ORIGIN + DOWN * 0.5, color=WHITE)
            eq_label = Text("Equilibrium", font_size=24).next_to(eq_point, DOWN)
            self.play(Create(axis), FadeIn(eq_point), Write(eq_label), run_time=min(3, tracker.duration))

        # --- SHM Parameters ---
        A = 3
        T = 6
        omega = 2 * np.pi / T
        t = ValueTracker(0)

        # Particle performing SHM
        particle = always_redraw(lambda: Dot(color=RED).move_to(np.array([A * np.cos(omega * t.get_value()), -0.5, 0])))
        velocity_arrow = always_redraw(lambda: Arrow(
            start=particle.get_center(),
            end=particle.get_center() + np.array([-0.8 * np.sin(omega * t.get_value()), 0, 0]),
            buff=0,
            color=BLUE
        ))
        self.add(particle, velocity_arrow)

        left_label = Text("−A", font_size=26, color=BLUE).next_to(LEFT * A + DOWN * 0.5, DOWN)
        right_label = Text("+A", font_size=26, color=BLUE).next_to(RIGHT * A + DOWN * 0.5, DOWN)
        self.play(Write(left_label), Write(right_label))

        # --- Energy bars setup ---
        bar_width = 0.4
        max_height = 1.5

        ke_bar = always_redraw(lambda: Rectangle(
            width=bar_width,
            height=max_height * np.sin(omega * t.get_value())**2,
            color=RED,
            fill_opacity=0.8
        ).next_to(LEFT * 1.0 + DOWN * (3.5 - max_height * np.sin(omega * t.get_value())**2 / 2), UP, buff=0))

        pe_bar = always_redraw(lambda: Rectangle(
            width=bar_width,
            height=max_height * np.cos(omega * t.get_value())**2,
            color=GREEN,
            fill_opacity=0.8
        ).next_to(RIGHT * 1.0 + DOWN * (3.5 - max_height * np.cos(omega * t.get_value())**2 / 2), UP, buff=0))

        ke_label = Text("KE", font_size=24, color=RED).next_to(LEFT * 1.0 + DOWN * 3.4, DOWN)
        pe_label = Text("PE", font_size=24, color=GREEN).next_to(RIGHT * 1.0 + DOWN * 3.4, DOWN)
        self.add(ke_bar, pe_bar, ke_label, pe_label)

        # --- Motion + narration ---
        with self.voiceover(text=storyboard["scenes"][1]["narration_segments"][0]["text"]) as tracker:
            self.play(t.animate.set_value(6 * np.pi), run_time=min(30, tracker.duration), rate_func=linear)

        # --- Mathematical explanation section ---
        for i, segment in enumerate(storyboard["scenes"][2]["narration_segments"]):
            text = segment["text"]

            # Switch visual stages based on narration index
            if i == 0:
                with self.voiceover(text=text) as tracker:
                    self.play(
                        FadeOut(axis), FadeOut(particle), FadeOut(velocity_arrow),
                        FadeOut(left_label), FadeOut(right_label),
                        FadeOut(ke_bar), FadeOut(pe_bar),
                        FadeOut(ke_label), FadeOut(pe_label),
                        FadeOut(eq_point), FadeOut(eq_label),
                        run_time=min(2, tracker.duration)
                    )
                    eq1 = MathTex("x = A \\cos(\\omega t)").scale(0.8).next_to(title, DOWN)
                    self.play(Write(eq1), run_time=min(3, tracker.duration))

            elif i == 1:
                with self.voiceover(text=text) as tracker:
                    eq1_1 = MathTex("\\omega = 2\\pi f").scale(0.8).next_to(eq1, RIGHT, buff=0.5).shift(RIGHT * 1.2)
                    self.play(Write(eq1_1), run_time=min(4, tracker.duration))
                    self.play(FadeOut(eq1_1))

            elif i == 2:
                with self.voiceover(text=text) as tracker:
                    eq1_2 = MathTex("= A \\cos(2 \\pi f t)").scale(0.8).next_to(eq1, RIGHT)
                    self.play(Write(eq1_2), run_time=min(3, tracker.duration))

            elif i == 3:
                with self.voiceover(text=text) as tracker:
                    eq2 = MathTex("\\frac{dx}{dt}= v = -2\\pi f A \\sin(2\\pi f t)").scale(0.8).next_to(eq1, DOWN)
                    self.play(Write(eq2), run_time=min(3, tracker.duration))

            elif i == 4:
                with self.voiceover(text=text) as tracker:
                    eq3 = MathTex("KE = \\tfrac{1}{2} m (2\\pi f A)^2 \\sin^2(2\\pi f t)").scale(0.8).next_to(eq2, DOWN)
                    self.play(Write(eq3), run_time=min(3, tracker.duration))

            elif i == 5:
                with self.voiceover(text=text) as tracker:
                    eq4 = MathTex("\\sin^2(\\theta) = \\tfrac{1 - \\cos(2\\theta)}{2}").scale(0.8).next_to(eq3, DOWN)
                    self.play(Write(eq4), run_time=min(4, tracker.duration))

            elif i == 6:
                with self.voiceover(text=text) as tracker:
                    eq5 = MathTex("KE \\propto \\frac{1 - \\cos(4\\pi f t)}{2}").scale(0.8).next_to(eq4, DOWN)
                    eq5_1 = MathTex("\\omega_{KE} = 2\\omega").scale(0.8).next_to(eq5, RIGHT, buff=0.5).shift(RIGHT * 1.2)
                    self.play(Write(eq5), Write(eq5_1), run_time=min(3, tracker.duration))

            elif i == 7:
                with self.voiceover(text=text) as tracker:
                    eq6 = MathTex("f_{KE} = \\frac{\\omega_{KE}}{2\\pi} = \\frac{4\\pi f}{2\\pi} = 2f").scale(0.8).next_to(eq5, DOWN)
                    self.play(Write(eq6), run_time=min(3, tracker.duration))
                    self.wait(2)

        # --- Final narrated conclusion ---
        with self.voiceover(text=storyboard["scenes"][3]["narration_segments"][0]["text"]) as tracker:
            final_text = Text("Kinetic Energy oscillates with frequency 2f", color=GREEN).scale(0.8).to_edge(DOWN)
            self.play(Write(final_text))
            self.wait(2)


Manim Community v0.19.0

[11/02/25 10:35:11] WARNING  Some options were not used: {'shortest': '1', 'metadata':     ]8;id=275534;file:///root/linux/linux_env/lib/python3.12/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=601081;file:///root/linux/linux_env/lib/python3.12/site-packages/manim/scene/scene_file_writer.py#801\801]8;;\
                             'comment=Rendered with Manim Community v0.19.0'}                                      